In [ ]:
# 1. Install dependencies
!pip install flask flask-cors langchain langchain-openai paddleocr openai pillow requests pyngrok
!pip uninstall -y paddlepaddle paddlepaddle-gpu
!pip install paddlepaddle -f https://www.paddlepaddle.org.cn/whl/mkl/avx/stable.html
!pip install "paddleocr>=2.6.1.3"

# 2. Set API key
import os
os.environ['OPENAI_API_KEY'] = ''

# 3. Optional: Set ngrok token for better performance
os.environ['NGROK_AUTHTOKEN'] = ''

In [ ]:
#!/usr/bin/env python3
"""
FastAPI Receipt OCR Parser for Google Colab with integrated ngrok and SQLite storage
Uses PaddleOCR Mobile + OpenAI for structured receipt parsing
"""

import os
import json
import logging
from typing import Dict, Any, List, Optional
from datetime import datetime
# Add these imports to your existing imports section
import sqlite3
import uuid
from contextlib import contextmanager
from pathlib import Path
import hashlib
import secrets
from typing import Dict, Any, List, Optional
from datetime import datetime, timedelta

# =============================================================================
# ENVIRONMENT CONFIGURATION - SET YOUR KEYS HERE
# =============================================================================
OPENAI_API_KEY = ""  # Replace with your OpenAI API key
NGROK_AUTH_TOKEN = ""  # Replace with your ngrok auth token
OPENAI_MODEL = "gpt-4o-mini"  # Default OpenAI model
DATABASE_PATH = "receipts.db"  # SQLite database file

# Set environment variables
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
# =============================================================================

import uvicorn
from fastapi import FastAPI, HTTPException, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel, HttpUrl
import requests
from PIL import Image
import io
import base64
import nest_asyncio
from pyngrok import ngrok
import sys
import numpy as np
# PaddleOCR and OpenAI imports
from paddleocr import PaddleOCR
import openai

# Enable nested asyncio for Colab
nest_asyncio.apply()

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    stream=sys.stdout,
    format='[%(levelname)s] %(asctime)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Initialize FastAPI app
app = FastAPI(
    title="Receipt OCR Parser API with Storage",
    description="Extract structured data from receipt images using PaddleOCR Mobile + OpenAI and store in SQLite",
    version="1.0.0"
)

# Add CORS middleware for cross-origin requests
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# =============================================================================
# PYDANTIC MODELS
# =============================================================================

class ProcessImageRequest(BaseModel):
    image_url: Optional[HttpUrl] = None
    image_base64: Optional[str] = None

class ReceiptData(BaseModel):
    merchant_name: Optional[str] = None
    merchant_address: Optional[str] = None
    merchant_phone: Optional[str] = None
    transaction_date: Optional[str] = None
    transaction_time: Optional[str] = None
    transaction_id: Optional[str] = None
    order_number: Optional[str] = None
    items: List[Dict[str, Any]] = []
    subtotal: Optional[float] = None
    tax_gst: Optional[float] = None
    tax_qst: Optional[float] = None
    total_tax: Optional[float] = None
    total_amount: Optional[float] = None
    payment_method: Optional[str] = None

class ReceiptDataWithConfidence(BaseModel):
    merchant_name: Optional[str] = None
    merchant_name_confidence: Optional[float] = None
    merchant_address: Optional[str] = None
    merchant_address_confidence: Optional[float] = None
    merchant_phone: Optional[str] = None
    merchant_phone_confidence: Optional[float] = None
    transaction_date: Optional[str] = None
    transaction_date_confidence: Optional[float] = None
    transaction_time: Optional[str] = None
    transaction_time_confidence: Optional[float] = None
    transaction_id: Optional[str] = None
    transaction_id_confidence: Optional[float] = None
    order_number: Optional[str] = None
    order_number_confidence: Optional[float] = None
    items: List[Dict[str, Any]] = []
    subtotal: Optional[float] = None
    subtotal_confidence: Optional[float] = None
    tax_gst: Optional[float] = None
    tax_gst_confidence: Optional[float] = None
    tax_qst: Optional[float] = None
    tax_qst_confidence: Optional[float] = None
    total_tax: Optional[float] = None
    total_tax_confidence: Optional[float] = None
    total_amount: Optional[float] = None
    total_amount_confidence: Optional[float] = None
    payment_method: Optional[str] = None
    payment_method_confidence: Optional[float] = None

class ProcessImageResponse(BaseModel):
    success: bool
    receipt_data: Optional[ReceiptDataWithConfidence] = None
    ocr_confidence_scores: List[float] = []
    average_confidence: float = 0.0
    raw_ocr_text: List[str] = []
    processing_time_seconds: float = 0.0
    error_message: Optional[str] = None

# Storage-related models
class StoreReceiptRequest(BaseModel):
    user_id: str  # Required field to link receipt to user
    receipt_data: ReceiptDataWithConfidence
    ocr_confidence_scores: List[float] = []
    average_confidence: float = 0.0
    raw_ocr_text: List[str] = []
    processing_time_seconds: float = 0.0
    image_url: Optional[str] = None
    notes: Optional[str] = None

class StoredReceiptResponse(BaseModel):
    success: bool
    receipt_id: Optional[str] = None
    message: str
    error_message: Optional[str] = None

class ReceiptSearchRequest(BaseModel):
    user_id: Optional[str] = None  # Filter by user
    merchant_name: Optional[str] = None
    date_from: Optional[str] = None  # YYYY-MM-DD format
    date_to: Optional[str] = None    # YYYY-MM-DD format
    min_amount: Optional[float] = None
    max_amount: Optional[float] = None
    limit: int = 50

class UpdateReceiptRequest(BaseModel):
    user_id: Optional[str] = None  # For authorization check
    receipt_data: Optional[ReceiptDataWithConfidence] = None
    notes: Optional[str] = None
    image_url: Optional[str] = None

class UpdateReceiptResponse(BaseModel):
    success: bool
    message: str
    error_message: Optional[str] = None

class DeleteReceiptRequest(BaseModel):
    user_id: Optional[str] = None  # For authorization check

class DeleteReceiptResponse(BaseModel):
    success: bool
    message: str
    error_message: Optional[str] = None

class Item(BaseModel):
    item_id: int
    receipt_id: str
    item_name: str = ""  # Default to empty string instead of None
    quantity: float = 0.0
    unit_price: float = 0.0
    total_price: float = 0.0
    item_order: int = 0

class SearchReceiptInfo(BaseModel):
    receipt_id: str
    user_id: str = ""  # Required but can be empty
    merchant_name: Optional[str] = None
    merchant_address: Optional[str] = None
    merchant_phone: Optional[str] = None
    transaction_date: Optional[str] = None
    transaction_time: Optional[str] = None
    transaction_id: Optional[str] = None
    order_number: Optional[str] = None
    subtotal: Optional[float] = None
    tax_gst: Optional[float] = None
    tax_qst: Optional[float] = None
    total_tax: Optional[float] = None
    total_amount: Optional[float] = None
    payment_method: Optional[str] = None
    average_confidence: Optional[float] = None
    processing_time_seconds: Optional[float] = None
    created_at: str
    notes: Optional[str] = None
    image_url: Optional[str] = None
    items: List[Item] = []
    raw_ocr_text: List[str] = []
    ocr_confidence_scores: List[float] = []

class StoredReceiptInfo(BaseModel):
    receipt_id: str
    merchant_name: Optional[str]
    transaction_date: Optional[str]
    total_amount: Optional[float]
    created_at: str
    notes: Optional[str]
    image_url: Optional[str]

class SearchReceiptsResponse(BaseModel):
    success: bool
    receipts: List[SearchReceiptInfo] = []
    count: int = 0
    error_message: Optional[str] = None

# User-related models
class User(BaseModel):
    user_id: str
    username: str
    email: str
    role: str  # 'admin', 'user'
    is_active: bool
    created_at: str
    last_login: Optional[str] = None

class CreateUserRequest(BaseModel):
    username: str
    email: str
    full_name: Optional[str] = None
    password: str
    role: str = "user"

class LoginRequest(BaseModel):
    username: str
    password: str

class LoginResponse(BaseModel):
    success: bool
    message: str
    user: Optional[User] = None
    session_token: Optional[str] = None
    error_message: Optional[str] = None

class UserResponse(BaseModel):
    success: bool
    user: Optional[User] = None
    message: str
    error_message: Optional[str] = None

class UsersListResponse(BaseModel):
    success: bool
    users: List[User] = []
    count: int = 0
    error_message: Optional[str] = None

# Global OCR instance (initialized once for efficiency)
ocr_instance = None

# =============================================================================
# DATABASE FUNCTIONS
# =============================================================================

@contextmanager
def get_db_connection():
    """Context manager for database connections"""
    conn = sqlite3.connect(DATABASE_PATH)
    conn.row_factory = sqlite3.Row  # Enable column access by name
    try:
        yield conn
        conn.commit()
    except Exception:
        conn.rollback()
        raise
    finally:
        conn.close()

def initialize_database():
    """Initialize SQLite database with required tables"""
    with get_db_connection() as conn:
        cursor = conn.cursor()

        # Create users table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS users (
                user_id TEXT PRIMARY KEY,
                username TEXT UNIQUE NOT NULL,
                email TEXT UNIQUE NOT NULL,
                full_name TEXT,
                password_hash TEXT NOT NULL,
                salt TEXT NOT NULL,
                role TEXT NOT NULL DEFAULT 'user',
                is_active INTEGER DEFAULT 1,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                last_login TIMESTAMP
            )
        ''')

        # Create receipts table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS receipts (
                receipt_id TEXT PRIMARY KEY,
                user_id TEXT,
                merchant_name TEXT,
                merchant_address TEXT,
                merchant_phone TEXT,
                transaction_date TEXT,
                transaction_time TEXT,
                transaction_id TEXT,
                order_number TEXT,
                subtotal REAL,
                tax_gst REAL,
                tax_qst REAL,
                total_tax REAL,
                total_amount REAL,
                payment_method TEXT,
                average_confidence REAL,
                processing_time_seconds REAL,
                image_url TEXT,
                notes TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                raw_ocr_text TEXT,  -- JSON string of OCR texts
                ocr_confidence_scores TEXT  -- JSON string of confidence scores
            )
        ''')

        # Create receipt_items table
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS receipt_items (
                item_id INTEGER PRIMARY KEY AUTOINCREMENT,
                receipt_id TEXT,
                item_name TEXT,
                quantity REAL,
                unit_price REAL,
                total_price REAL,
                item_order INTEGER,
                FOREIGN KEY (receipt_id) REFERENCES receipts (receipt_id) ON DELETE CASCADE
            )
        ''')



        # Create indexes for better query performance
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_users_username ON users(username)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_users_email ON users(email)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_receipts_merchant ON receipts(merchant_name)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_receipts_date ON receipts(transaction_date)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_receipts_amount ON receipts(total_amount)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_receipts_created ON receipts(created_at)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_receipts_user ON receipts(user_id)')

        logger.info("Database initialized successfully")

# If you have an existing database, you might need to add the column manually:
def add_user_id_column_if_missing():
    """Add user_id column to existing receipts table if it doesn't exist"""
    with get_db_connection() as conn:
        cursor = conn.cursor()

        # Check if user_id column exists
        cursor.execute("PRAGMA table_info(receipts)")
        columns = [column[1] for column in cursor.fetchall()]

        if 'user_id' not in columns:
            logger.info("Adding user_id column to receipts table...")
            cursor.execute('ALTER TABLE receipts ADD COLUMN user_id TEXT')
            logger.info("user_id column added successfully")
        else:
            logger.info("user_id column already exists")

# =============================================================================
# USER MANAGEMENT FUNCTIONS
# =============================================================================

def hash_password(password: str) -> tuple[str, str]:
    """Hash password with salt"""
    salt = secrets.token_hex(32)
    password_hash = hashlib.pbkdf2_hmac('sha256', password.encode('utf-8'), salt.encode('utf-8'), 100000)
    return password_hash.hex(), salt

def verify_password(password: str, password_hash: str, salt: str) -> bool:
    """Verify password against hash"""
    computed_hash = hashlib.pbkdf2_hmac('sha256', password.encode('utf-8'), salt.encode('utf-8'), 100000)
    return computed_hash.hex() == password_hash

def create_default_admin_user():
    """Create default admin user if it doesn't exist"""
    try:
        with get_db_connection() as conn:
            cursor = conn.cursor()

            # Check if admin user already exists
            cursor.execute('SELECT user_id FROM users WHERE username = ? OR role = ?', ('admin', 'admin'))
            if cursor.fetchone():
                return  # Admin user already exists

            # Create default admin user
            user_id = str(uuid.uuid4())
            username = "admin"
            email = "admin@receipt-parser.com"
            password = "admin123"
            role = "admin"

            password_hash, salt = hash_password(password)

            cursor.execute('''
                INSERT INTO users (user_id, username, email, full_name, password_hash, salt, role, is_active, created_at)
                VALUES (?, ?, ?, ?, ?, ?, ?, 1, datetime('now'))
            ''', (user_id, username, email, "Administrator", password_hash, salt, role))

            logger.info("Default admin user created - Username: admin, Password: admin123")
            print("🔑 Default admin user created:")
            print("   Username: admin")
            print("   Password: admin123")
            print("   ⚠️  Please change the password after first login!")

    except Exception as e:
        logger.error(f"Failed to create default admin user: {e}")

def create_user(username: str, email: str, password: str, full_name: str = None, role: str = "user") -> str:
    """Create a new user"""
    try:
        with get_db_connection() as conn:
            cursor = conn.cursor()

            # Check if username or email already exists
            cursor.execute('SELECT user_id FROM users WHERE username = ? OR email = ?', (username, email))
            if cursor.fetchone():
                raise ValueError("Username or email already exists")

            user_id = str(uuid.uuid4())
            password_hash, salt = hash_password(password)

            cursor.execute('''
                INSERT INTO users (user_id, username, email, full_name, password_hash, salt, role, is_active, created_at)
                VALUES (?, ?, ?, ?, ?, ?, ?, 1, datetime('now'))
            ''', (user_id, username, email, full_name, password_hash, salt, role))

            logger.info(f"User created successfully: {username}")
            return user_id

    except Exception as e:
        logger.error(f"Failed to create user: {e}")
        raise

def authenticate_user(username: str, password: str) -> Optional[Dict[str, Any]]:
    """Authenticate user and return user data"""
    try:
        with get_db_connection() as conn:
            cursor = conn.cursor()

            cursor.execute('''
                SELECT user_id, username, email, full_name, password_hash, salt, role, is_active
                FROM users WHERE username = ? AND is_active = 1
            ''', (username,))

            user_row = cursor.fetchone()
            if not user_row:
                return None

            user_data = dict(user_row)

            # Verify password
            if not verify_password(password, user_data['password_hash'], user_data['salt']):
                return None

            # Update last login
            cursor.execute('''
                UPDATE users SET last_login = datetime('now') WHERE user_id = ?
            ''', (user_data['user_id'],))

            # Remove sensitive data
            del user_data['password_hash']
            del user_data['salt']

            # Convert timestamps to strings if they're not already
            if user_data.get('created_at'):
                user_data['created_at'] = str(user_data['created_at'])
            if user_data.get('last_login'):
                user_data['last_login'] = str(user_data['last_login'])

            return user_data

    except Exception as e:
        logger.error(f"Authentication failed: {e}")
        return None

# def create_session(user_id: str) -> str:
#     """Create a new session for user"""
#     try:
#         with get_db_connection() as conn:
#             cursor = conn.cursor()

#             session_id = secrets.token_urlsafe(32)
#             expires_at = datetime.now() + timedelta(days=7)  # 7 days expiry

#             cursor.execute('''
#                 INSERT INTO user_sessions (session_id, user_id, expires_at, is_active)
#                 VALUES (?, ?, ?, 1)
#             ''', (session_id, user_id, expires_at))

#             return session_id

#     except Exception as e:
#         logger.error(f"Failed to create session: {e}")
#         raise

# def get_user_by_session(session_id: str) -> Optional[Dict[str, Any]]:
#     """Get user data by session ID"""
#     try:
#         with get_db_connection() as conn:
#             cursor = conn.cursor()

#             cursor.execute('''
#                 SELECT u.user_id, u.username, u.email, u.role, u.is_active, u.created_at, u.last_login
#                 FROM users u
#                 JOIN user_sessions s ON u.user_id = s.user_id
#                 WHERE s.session_id = ? AND s.is_active = 1 AND s.expires_at > CURRENT_TIMESTAMP
#             ''', (session_id,))

#             user_row = cursor.fetchone()
#             if user_row:
#                 return dict(user_row)
#             return None

#     except Exception as e:
#         logger.error(f"Failed to get user by session: {e}")
#         return None

def get_user_by_id(user_id: str) -> Optional[Dict[str, Any]]:
    """Get user by ID with proper timestamp handling"""
    try:
        with get_db_connection() as conn:
            cursor = conn.cursor()

            cursor.execute('''
                SELECT user_id, username, email, full_name, role, is_active, created_at, last_login
                FROM users WHERE user_id = ?
            ''', (user_id,))

            user_row = cursor.fetchone()
            if not user_row:
                return None

            user_data = dict(user_row)

            # Convert timestamps to strings
            if user_data.get('created_at'):
                user_data['created_at'] = str(user_data['created_at'])
            if user_data.get('last_login'):
                user_data['last_login'] = str(user_data['last_login'])

            return user_data

    except Exception as e:
        logger.error(f"Failed to get user by ID: {e}")
        return None

def get_all_users() -> List[Dict[str, Any]]:
    """Get all users with proper timestamp handling"""
    try:
        with get_db_connection() as conn:
            cursor = conn.cursor()

            cursor.execute('''
                SELECT user_id, username, email, full_name, role, is_active, created_at, last_login
                FROM users
                ORDER BY created_at DESC
            ''')

            users = []
            for row in cursor.fetchall():
                user_data = dict(row)

                # Convert timestamps to strings
                if user_data.get('created_at'):
                    user_data['created_at'] = str(user_data['created_at'])
                if user_data.get('last_login'):
                    user_data['last_login'] = str(user_data['last_login'])

                users.append(user_data)

            return users

    except Exception as e:
        logger.error(f"Failed to get users: {e}")
        raise



# =============================================================================
# RECEIPTS MANAGEMENT FUNCTIONS
# =============================================================================
def store_receipt_to_db(
    user_id: str,  # Now required parameter
    receipt_data: ReceiptDataWithConfidence,
    ocr_confidence_scores: List[float],
    raw_ocr_text: List[str],
    average_confidence: float,
    processing_time_seconds: float,
    image_url: Optional[str] = None,
    notes: Optional[str] = None
) -> str:
    """Store receipt data to SQLite database"""

    receipt_id = str(uuid.uuid4())

    try:
        with get_db_connection() as conn:
            cursor = conn.cursor()

            # First, verify the user exists
            cursor.execute('SELECT user_id FROM users WHERE user_id = ? AND is_active = 1', (user_id,))
            if not cursor.fetchone():
                raise ValueError(f"User with ID {user_id} not found or inactive")

            # Insert main receipt record
            cursor.execute('''
                INSERT INTO receipts (
                    receipt_id, user_id, merchant_name, merchant_address, merchant_phone,
                    transaction_date, transaction_time, transaction_id, order_number,
                    subtotal, tax_gst, tax_qst, total_tax, total_amount, payment_method,
                    average_confidence, processing_time_seconds, image_url, notes,
                    raw_ocr_text, ocr_confidence_scores
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                receipt_id,
                user_id,
                receipt_data.merchant_name,
                receipt_data.merchant_address,
                receipt_data.merchant_phone,
                receipt_data.transaction_date,
                receipt_data.transaction_time,
                receipt_data.transaction_id,
                receipt_data.order_number,
                receipt_data.subtotal,
                receipt_data.tax_gst,
                receipt_data.tax_qst,
                receipt_data.total_tax,
                receipt_data.total_amount,
                receipt_data.payment_method,
                average_confidence,
                processing_time_seconds,
                image_url,
                notes,
                json.dumps(raw_ocr_text),
                json.dumps(ocr_confidence_scores)
            ))

            # Insert receipt items
            for idx, item in enumerate(receipt_data.items):
                cursor.execute('''
                    INSERT INTO receipt_items (
                        receipt_id, item_name, quantity, unit_price, total_price, item_order
                    ) VALUES (?, ?, ?, ?, ?, ?)
                ''', (
                    receipt_id,
                    item.get('name'),
                    item.get('quantity'),
                    item.get('unit_price'),
                    item.get('total_price'),
                    idx
                ))



            logger.info(f"Receipt stored successfully with ID: {receipt_id}")
            return receipt_id

    except Exception as e:
        logger.error(f"Failed to store receipt to database: {e}")
        raise

def search_receipts(
    user_id: Optional[str] = None,
    merchant_name: Optional[str] = None,
    date_from: Optional[str] = None,
    date_to: Optional[str] = None,
    min_amount: Optional[float] = None,
    max_amount: Optional[float] = None,
    limit: int = 50
) -> List[Dict[str, Any]]:
    """Search receipts based on criteria - now includes items via JOIN"""

    try:
        with get_db_connection() as conn:
            cursor = conn.cursor()

            # Build dynamic query conditions
            conditions = []
            params = []

            if merchant_name:
                conditions.append("r.merchant_name LIKE ?")
                params.append(f"%{merchant_name}%")

            if date_from:
                conditions.append("r.transaction_date >= ?")
                params.append(date_from)

            if date_to:
                conditions.append("r.transaction_date <= ?")
                params.append(date_to)

            if min_amount is not None:
                conditions.append("r.total_amount >= ?")
                params.append(min_amount)

            if max_amount is not None:
                conditions.append("r.total_amount <= ?")
                params.append(max_amount)

            if user_id:
                conditions.append("r.user_id = ?")
                params.append(user_id)

            where_clause = ""
            if conditions:
                where_clause = "WHERE " + " AND ".join(conditions)

            # MAIN QUERY: Get receipts first (without items to avoid duplication)
            receipts_query = f'''
                SELECT r.receipt_id, r.user_id, r.merchant_name, r.merchant_address, r.merchant_phone,
                       r.transaction_date, r.transaction_time, r.transaction_id, r.order_number,
                       r.subtotal, r.tax_gst, r.tax_qst, r.total_tax, r.total_amount, r.payment_method,
                       r.average_confidence, r.processing_time_seconds, r.image_url, r.notes,
                       r.created_at, r.raw_ocr_text, r.ocr_confidence_scores
                FROM receipts r
                {where_clause}
                ORDER BY r.created_at DESC
                LIMIT ?
            '''

            params.append(limit)
            cursor.execute(receipts_query, params)
            receipt_rows = cursor.fetchall()

            receipts = []
            for receipt_row in receipt_rows:
                receipt_data = dict(receipt_row)

                # Parse JSON fields if they exist and are not None
                raw_ocr_text = []
                ocr_confidence_scores = []

                if receipt_data.get('raw_ocr_text'):
                    try:
                        raw_ocr_text = json.loads(receipt_data['raw_ocr_text'])
                        # Filter out empty strings but keep the list structure
                        if not isinstance(raw_ocr_text, list):
                            raw_ocr_text = []
                    except (json.JSONDecodeError, TypeError):
                        logger.warning(f"Failed to parse raw_ocr_text for receipt {receipt_data['receipt_id']}")
                        raw_ocr_text = []

                if receipt_data.get('ocr_confidence_scores'):
                    try:
                        ocr_confidence_scores = json.loads(receipt_data['ocr_confidence_scores'])
                        if not isinstance(ocr_confidence_scores, list):
                            ocr_confidence_scores = []
                    except (json.JSONDecodeError, TypeError):
                        logger.warning(f"Failed to parse ocr_confidence_scores for receipt {receipt_data['receipt_id']}")
                        ocr_confidence_scores = []

                # Get items for this specific receipt
                cursor.execute('''
                    SELECT item_id, item_name, quantity, unit_price, total_price, item_order
                    FROM receipt_items
                    WHERE receipt_id = ?
                    ORDER BY item_order ASC
                ''', (receipt_data['receipt_id'],))

                items_rows = cursor.fetchall()
                items = []
                for item_row in items_rows:
                    item_dict = dict(item_row)
                    # Create Item object structure that matches the Pydantic model
                    items.append({
                        'item_id': item_dict.get('item_id', 0),
                        'receipt_id': receipt_data['receipt_id'],
                        'item_name': item_dict.get('item_name') or '',
                        'quantity': float(item_dict.get('quantity', 0.0)) if item_dict.get('quantity') is not None else 0.0,
                        'unit_price': float(item_dict.get('unit_price', 0.0)) if item_dict.get('unit_price') is not None else 0.0,
                        'total_price': float(item_dict.get('total_price', 0.0)) if item_dict.get('total_price') is not None else 0.0,
                        'item_order': int(item_dict.get('item_order', 0)) if item_dict.get('item_order') is not None else 0
                    })

                # Structure the receipt data to match SearchReceiptInfo model
                structured_receipt = {
                    'receipt_id': receipt_data['receipt_id'],
                    'user_id': receipt_data.get('user_id') or '',
                    'merchant_name': receipt_data.get('merchant_name'),
                    'merchant_address': receipt_data.get('merchant_address'),
                    'merchant_phone': receipt_data.get('merchant_phone'),
                    'transaction_date': receipt_data.get('transaction_date'),
                    'transaction_time': receipt_data.get('transaction_time'),
                    'transaction_id': receipt_data.get('transaction_id'),
                    'order_number': receipt_data.get('order_number'),
                    'subtotal': float(receipt_data.get('subtotal', 0.0)) if receipt_data.get('subtotal') is not None else None,
                    'tax_gst': float(receipt_data.get('tax_gst', 0.0)) if receipt_data.get('tax_gst') is not None else None,
                    'tax_qst': float(receipt_data.get('tax_qst', 0.0)) if receipt_data.get('tax_qst') is not None else None,
                    'total_tax': float(receipt_data.get('total_tax', 0.0)) if receipt_data.get('total_tax') is not None else None,
                    'total_amount': float(receipt_data.get('total_amount', 0.0)) if receipt_data.get('total_amount') is not None else None,
                    'payment_method': receipt_data.get('payment_method'),
                    'average_confidence': float(receipt_data.get('average_confidence', 0.0)) if receipt_data.get('average_confidence') is not None else None,
                    'processing_time_seconds': float(receipt_data.get('processing_time_seconds', 0.0)) if receipt_data.get('processing_time_seconds') is not None else None,
                    'created_at': str(receipt_data.get('created_at', '')),
                    'notes': receipt_data.get('notes'),
                    'image_url': receipt_data.get('image_url'),
                    'items': items,
                    'raw_ocr_text': raw_ocr_text,
                    'ocr_confidence_scores': ocr_confidence_scores
                }

                receipts.append(structured_receipt)

            logger.info(f"Found {len(receipts)} receipts with items")
            return receipts

    except Exception as e:
        logger.error(f"Failed to search receipts: {e}")
        raise


def delete_receipt_from_db(receipt_id: str, user_id: Optional[str] = None) -> bool:
    """Delete receipt from SQLite database"""
    try:
        with get_db_connection() as conn:
            cursor = conn.cursor()

            # First, check if receipt exists and get current user_id
            cursor.execute('SELECT user_id FROM receipts WHERE receipt_id = ?', (receipt_id,))
            receipt_row = cursor.fetchone()

            if not receipt_row:
                raise ValueError("Receipt not found")

            current_user_id = receipt_row['user_id']

            # If user_id is provided, check authorization
            if user_id and current_user_id != user_id:
                raise ValueError("Access denied - receipt belongs to different user")

            # Delete receipt items first (due to foreign key constraint)
            cursor.execute('DELETE FROM receipt_items WHERE receipt_id = ?', (receipt_id,))
            deleted_items = cursor.rowcount

            # Delete the main receipt record
            cursor.execute('DELETE FROM receipts WHERE receipt_id = ?', (receipt_id,))
            deleted_receipts = cursor.rowcount

            if deleted_receipts == 0:
                raise ValueError("No receipt was deleted")

            logger.info(f"Receipt {receipt_id} deleted successfully (removed {deleted_items} items)")
            return True

    except Exception as e:
        logger.error(f"Failed to delete receipt from database: {e}")
        raise


def get_receipt_by_id(receipt_id: str) -> Optional[Dict[str, Any]]:
    """Get full receipt data by ID"""

    try:
        with get_db_connection() as conn:
            cursor = conn.cursor()

            # Get main receipt data
            cursor.execute('SELECT * FROM receipts WHERE receipt_id = ?', (receipt_id,))
            receipt_row = cursor.fetchone()

            if not receipt_row:
                return None

            # Get receipt items
            cursor.execute('''
                SELECT item_name, quantity, unit_price, total_price
                FROM receipt_items
                WHERE receipt_id = ?
                ORDER BY item_order
            ''', (receipt_id,))
            items_rows = cursor.fetchall()

            # Convert to dict
            receipt_data = dict(receipt_row)
            receipt_data['items'] = [dict(row) for row in items_rows]

            # Parse JSON fields
            if receipt_data['raw_ocr_text']:
                receipt_data['raw_ocr_text'] = json.loads(receipt_data['raw_ocr_text'])
            if receipt_data['ocr_confidence_scores']:
                receipt_data['ocr_confidence_scores'] = json.loads(receipt_data['ocr_confidence_scores'])

            return receipt_data

    except Exception as e:
        logger.error(f"Failed to get receipt by ID: {e}")
        raise

def update_receipt_in_db(
    receipt_id: str,
    user_id: Optional[str] = None,
    receipt_data: Optional[ReceiptDataWithConfidence] = None,
    notes: Optional[str] = None,
    image_url: Optional[str] = None
) -> bool:
    """Update receipt data in SQLite database"""
    try:
        with get_db_connection() as conn:
            cursor = conn.cursor()

            # First, check if receipt exists and get current user_id
            cursor.execute('SELECT user_id FROM receipts WHERE receipt_id = ?', (receipt_id,))
            receipt_row = cursor.fetchone()

            if not receipt_row:
                raise ValueError("Receipt not found")

            current_user_id = receipt_row['user_id']

            # If user_id is provided, check authorization
            if user_id and current_user_id != user_id:
                raise ValueError("Access denied - receipt belongs to different user")

            # Build update query dynamically based on provided fields
            update_fields = []
            params = []

            if receipt_data:
                # Update main receipt fields
                if receipt_data.merchant_name is not None:
                    update_fields.append("merchant_name = ?")
                    params.append(receipt_data.merchant_name)

                if receipt_data.merchant_address is not None:
                    update_fields.append("merchant_address = ?")
                    params.append(receipt_data.merchant_address)

                if receipt_data.merchant_phone is not None:
                    update_fields.append("merchant_phone = ?")
                    params.append(receipt_data.merchant_phone)

                if receipt_data.transaction_date is not None:
                    update_fields.append("transaction_date = ?")
                    params.append(receipt_data.transaction_date)

                if receipt_data.transaction_time is not None:
                    update_fields.append("transaction_time = ?")
                    params.append(receipt_data.transaction_time)

                if receipt_data.transaction_id is not None:
                    update_fields.append("transaction_id = ?")
                    params.append(receipt_data.transaction_id)

                if receipt_data.order_number is not None:
                    update_fields.append("order_number = ?")
                    params.append(receipt_data.order_number)

                if receipt_data.subtotal is not None:
                    update_fields.append("subtotal = ?")
                    params.append(receipt_data.subtotal)

                if receipt_data.tax_gst is not None:
                    update_fields.append("tax_gst = ?")
                    params.append(receipt_data.tax_gst)

                if receipt_data.tax_qst is not None:
                    update_fields.append("tax_qst = ?")
                    params.append(receipt_data.tax_qst)

                if receipt_data.total_tax is not None:
                    update_fields.append("total_tax = ?")
                    params.append(receipt_data.total_tax)

                if receipt_data.total_amount is not None:
                    update_fields.append("total_amount = ?")
                    params.append(receipt_data.total_amount)

                if receipt_data.payment_method is not None:
                    update_fields.append("payment_method = ?")
                    params.append(receipt_data.payment_method)

                # Update items if provided
                print(f"-----------------------------------: ", receipt_data.items)
                if receipt_data.items is not None:
                    # Delete existing items
                    cursor.execute('DELETE FROM receipt_items WHERE receipt_id = ?', (receipt_id,))

                    # Insert new items
                    for idx, item in enumerate(receipt_data.items):
                        cursor.execute('''
                            INSERT INTO receipt_items (
                                receipt_id, item_name, quantity, unit_price, total_price, item_order
                            ) VALUES (?, ?, ?, ?, ?, ?)
                        ''', (
                            receipt_id,
                            item.get('name'),
                            item.get('quantity'),
                            item.get('unit_price'),
                            item.get('total_price'),
                            idx
                        ))

            if notes is not None:
                update_fields.append("notes = ?")
                params.append(notes)

            if image_url is not None:
                update_fields.append("image_url = ?")
                params.append(image_url)

            # If no fields to update, return success
            if not update_fields:
                return True

            # Execute update query
            params.append(receipt_id)  # Add receipt_id for WHERE clause
            update_query = f'''
                UPDATE receipts
                SET {", ".join(update_fields)}
                WHERE receipt_id = ?
            '''

            cursor.execute(update_query, params)

            if cursor.rowcount == 0:
                raise ValueError("No receipt was updated")

            logger.info(f"Receipt {receipt_id} updated successfully")
            return True

    except Exception as e:
        logger.error(f"Failed to update receipt in database: {e}")
        raise
# =============================================================================
# EXISTING OCR FUNCTIONS (unchanged)
# =============================================================================

def setup_ngrok():
    """Set up ngrok tunnel"""
    try:
        # Set ngrok auth token
        ngrok.set_auth_token(NGROK_AUTH_TOKEN)

        # Create tunnel
        public_url = ngrok.connect(8000)
        logger.info(f"🌐 Public URL: {public_url}")
        print(f"\n" + "="*50)
        print(f"🚀 API is now accessible at: {public_url}")
        print(f"📋 Test endpoint: {public_url}/")
        print(f"🔧 Process endpoint: {public_url}/process-image")
        print(f"💾 Store endpoint: {public_url}/store-receipt")
        print(f"🔍 Search endpoint: {public_url}/search-receipts")
        print("="*50 + "\n")

        return public_url
    except Exception as e:
        logger.error(f"Failed to setup ngrok: {e}")
        raise

def initialize_ocr():
    """Initialize PaddleOCR Mobile instance"""
    global ocr_instance
    if ocr_instance is None:
        logger.info("Initializing PaddleOCR Mobile...")
        ocr_instance = PaddleOCR(
            text_detection_model_name="PP-OCRv5_mobile_det",
            text_recognition_model_name="PP-OCRv5_mobile_rec",
            use_doc_orientation_classify=False,
            use_doc_unwarping=False,
            use_textline_orientation=False
        )
        logger.info("PaddleOCR Mobile initialized successfully")
    return ocr_instance

def load_image_from_url(image_url: str) -> np.ndarray:
    """Download and load image from URL"""
    try:
        response = requests.get(str(image_url), timeout=30)
        response.raise_for_status()

        image = Image.open(io.BytesIO(response.content))
        # Convert PIL image to numpy array (RGB format)
        image_array = np.array(image)

        # If image is grayscale, convert to RGB
        if len(image_array.shape) == 2:
            image_array = np.stack([image_array] * 3, axis=-1)
        # If image has alpha channel, remove it
        elif image_array.shape[2] == 4:
            image_array = image_array[:, :, :3]

        return image_array

    except Exception as e:
        logger.error(f"Failed to load image from URL {image_url}: {e}")
        raise HTTPException(status_code=400, detail=f"Failed to load image from URL: {str(e)}")

def load_image_from_base64(base64_string: str) -> np.ndarray:
    """Decode and load image from base64 string"""
    try:
        # Handle data URL format (e.g., "data:image/jpeg;base64,...")
        if base64_string.startswith('data:'):
            # Extract the base64 part after the comma
            base64_string = base64_string.split(',')[1]

        # Decode base64
        image_data = base64.b64decode(base64_string)

        # Open image with PIL
        image = Image.open(io.BytesIO(image_data))

        # Convert PIL image to numpy array (RGB format)
        image_array = np.array(image)

        # If image is grayscale, convert to RGB
        if len(image_array.shape) == 2:
            image_array = np.stack([image_array] * 3, axis=-1)
        # If image has alpha channel, remove it
        elif image_array.shape[2] == 4:
            image_array = image_array[:, :, :3]

        return image_array

    except Exception as e:
        logger.error(f"Failed to decode base64 image: {e}")
        raise HTTPException(status_code=400, detail=f"Failed to decode base64 image: {str(e)}")

def create_openai_prompt(ocr_texts: List[str], ocr_scores: List[float]) -> str:
    """Create the OpenAI prompt with OCR data"""

    # Combine texts with their confidence scores
    ocr_text_with_scores = []
    for text, score in zip(ocr_texts, ocr_scores):
        ocr_text_with_scores.append(f'"{text}" (confidence: {score:.3f})')

    ocr_text_formatted = '\n'.join(ocr_text_with_scores)

    # Define the JSON schema for structured output with field mapping
    schema = {
        "type": "object",
        "properties": {
            "merchant_name": {"type": ["string", "null"]},
            "merchant_name_source_text": {"type": ["string", "null"], "description": "Exact OCR text used for merchant_name"},
            "merchant_address": {"type": ["string", "null"]},
            "merchant_address_source_text": {"type": ["string", "null"], "description": "Exact OCR text used for merchant_address"},
            "merchant_phone": {"type": ["string", "null"]},
            "merchant_phone_source_text": {"type": ["string", "null"], "description": "Exact OCR text used for merchant_phone"},
            "transaction_date": {"type": ["string", "null"]},
            "transaction_date_source_text": {"type": ["string", "null"], "description": "Exact OCR text used for transaction_date"},
            "transaction_time": {"type": ["string", "null"]},
            "transaction_time_source_text": {"type": ["string", "null"], "description": "Exact OCR text used for transaction_time"},
            "transaction_id": {"type": ["string", "null"]},
            "transaction_id_source_text": {"type": ["string", "null"], "description": "Exact OCR text used for transaction_id"},
            "order_number": {"type": ["string", "null"]},
            "order_number_source_text": {"type": ["string", "null"], "description": "Exact OCR text used for order_number"},
            "items": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "name": {"type": "string"},
                        "name_source_text": {"type": ["string", "null"], "description": "Exact OCR text used for item name"},
                        "quantity": {"type": ["number", "null"]},
                        "unit_price": {"type": ["number", "null"]},
                        "total_price": {"type": ["number", "null"]},
                        "total_price_source_text": {"type": ["string", "null"], "description": "Exact OCR text used for item total price"}
                    }
                }
            },
            "subtotal": {"type": ["number", "null"]},
            "subtotal_source_text": {"type": ["string", "null"], "description": "Exact OCR text used for subtotal"},
            "tax_gst": {"type": ["number", "null"]},
            "tax_gst_source_text": {"type": ["string", "null"], "description": "Exact OCR text used for GST tax"},
            "tax_qst": {"type": ["number", "null"]},
            "tax_qst_source_text": {"type": ["string", "null"], "description": "Exact OCR text used for QST tax"},
            "total_tax": {"type": ["number", "null"]},
            "total_tax_source_text": {"type": ["string", "null"], "description": "Exact OCR text used for total tax"},
            "total_amount": {"type": ["number", "null"]},
            "total_amount_source_text": {"type": ["string", "null"], "description": "Exact OCR text used for total amount"},
            "payment_method": {"type": ["string", "null"]},
            "payment_method_source_text": {"type": ["string", "null"], "description": "Exact OCR text used for payment method"}
        }
    }

    prompt = f"""You are an expert receipt parser. Analyze the following OCR text from a receipt and extract structured information.

OCR TEXT WITH CONFIDENCE SCORES:
{ocr_text_formatted}

OCR Configuration Used: PP-OCRv5_mobile (PaddleOCR Mobile)

CRITICAL PARSING RULES:
1. RECEIPT STRUCTURE: Follow typical receipt layout:
   - Merchant name is usually at the top (e.g., "HURLEY'S")
   - Address/location appears RIGHT AFTER merchant name (e.g., "AEROPORT DE MONTREAL")
   - Phone number comes after address
   - Transaction details (order numbers, dates) come in the middle
   - Items and prices are in the main body
   - Taxes and totals are at the bottom

2. DATE/TIME FORMAT: "Ma121'2505:03Ap" means May 12, 2025 at 05:03 AM
   - Ma = May, 121 = 12th day, '25 = 2025, 05:03 = time, Ap = AM

3. DO NOT confuse transaction IDs, order numbers, or reference codes with addresses

4. Extract ALL numerical values as numbers, not strings

5. For taxes: GST/TPS is goods and services tax, QST/TVQ is Quebec sales tax

6. Parse items carefully - look for item names followed by prices

7. IMPORTANT: For each extracted field, also provide the "source_text" field with the EXACT OCR text that was used to extract that information. This will help map confidence scores.

Please extract the receipt information and return it as a JSON object that matches this schema:
{json.dumps(schema, indent=2)}

Return ONLY the JSON object, no additional text or explanation."""

    return prompt

def map_confidence_scores(parsed_data: Dict[str, Any], ocr_texts: List[str], ocr_scores: List[float]) -> ReceiptDataWithConfidence:
    """Map confidence scores to extracted fields based on source text"""

    # Create a mapping from OCR text to confidence score, filtering out empty strings
    text_to_confidence = {}
    for text, score in zip(ocr_texts, ocr_scores):
        if text and text.strip():  # Only map non-empty strings
            text_to_confidence[text] = score

    def get_confidence_for_field(source_text_key: str) -> Optional[float]:
        """Get confidence score for a field based on its source text"""
        if source_text_key in parsed_data and parsed_data[source_text_key]:
            source_text = parsed_data[source_text_key]
            # Try exact match first
            if source_text in text_to_confidence:
                return text_to_confidence[source_text]
            # Try partial match (sometimes OCR text might be slightly different)
            for ocr_text, confidence in text_to_confidence.items():
                if source_text.lower() in ocr_text.lower() or ocr_text.lower() in source_text.lower():
                    return confidence
        return None

    # Map items with confidence scores
    items_with_confidence = []
    for item in parsed_data.get('items', []):
        item_with_conf = {
            'name': item.get('name'),
            'quantity': item.get('quantity'),
            'unit_price': item.get('unit_price'),
            'total_price': item.get('total_price')
        }

        # Add confidence scores for item fields
        if 'name_source_text' in item and item['name_source_text']:
            name_confidence = None
            for ocr_text, confidence in text_to_confidence.items():
                if item['name_source_text'].lower() in ocr_text.lower() or ocr_text.lower() in item['name_source_text'].lower():
                    name_confidence = confidence
                    break
            item_with_conf['name_confidence'] = name_confidence

        if 'total_price_source_text' in item and item['total_price_source_text']:
            price_confidence = text_to_confidence.get(item['total_price_source_text'])
            item_with_conf['total_price_confidence'] = price_confidence

        items_with_confidence.append(item_with_conf)
    # Format transaction_date before creating the response
    formatted_transaction_date = parsed_data.get('transaction_date')
    if formatted_transaction_date:
        iso_date = format_transaction_date(formatted_transaction_date)
        if iso_date:
            formatted_transaction_date = iso_date
        else:
            print(f"Warning: Could not format transaction_date: {formatted_transaction_date}")

    # Create ReceiptDataWithConfidence object
    receipt_data = ReceiptDataWithConfidence(
        merchant_name=parsed_data.get('merchant_name'),
        merchant_name_confidence=get_confidence_for_field('merchant_name_source_text'),
        merchant_address=parsed_data.get('merchant_address'),
        merchant_address_confidence=get_confidence_for_field('merchant_address_source_text'),
        merchant_phone=parsed_data.get('merchant_phone'),
        merchant_phone_confidence=get_confidence_for_field('merchant_phone_source_text'),
        transaction_date=parsed_data.get('transaction_date'),
        transaction_date_confidence=get_confidence_for_field('transaction_date_source_text'),
        transaction_time=parsed_data.get('transaction_time'),
        transaction_time_confidence=get_confidence_for_field('transaction_time_source_text'),
        transaction_id=parsed_data.get('transaction_id'),
        transaction_id_confidence=get_confidence_for_field('transaction_id_source_text'),
        order_number=parsed_data.get('order_number'),
        order_number_confidence=get_confidence_for_field('order_number_source_text'),
        items=items_with_confidence,
        subtotal=parsed_data.get('subtotal'),
        subtotal_confidence=get_confidence_for_field('subtotal_source_text'),
        tax_gst=parsed_data.get('tax_gst'),
        tax_gst_confidence=get_confidence_for_field('tax_gst_source_text'),
        tax_qst=parsed_data.get('tax_qst'),
        tax_qst_confidence=get_confidence_for_field('tax_qst_source_text'),
        total_tax=parsed_data.get('total_tax'),
        total_tax_confidence=get_confidence_for_field('total_tax_source_text'),
        total_amount=parsed_data.get('total_amount'),
        total_amount_confidence=get_confidence_for_field('total_amount_source_text'),
        payment_method=parsed_data.get('payment_method'),
        payment_method_confidence=get_confidence_for_field('payment_method_source_text')
    )

    return receipt_data

async def call_openai_api(prompt: str) -> Dict[str, Any]:
    """Call OpenAI API to parse receipt data"""
    try:
        client = openai.OpenAI(api_key=OPENAI_API_KEY)

        response = client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[
                {
                    "role": "system",
                    "content": "You are an expert receipt parser. Always return valid JSON."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            temperature=0.1,
            max_tokens=2000
        )

        content = response.choices[0].message.content.strip()

        # Try to extract JSON from the response
        try:
            # Remove markdown code blocks if present
            if content.startswith('```json'):
                content = content[7:]
            if content.endswith('```'):
                content = content[:-3]

            parsed_data = json.loads(content)
            return parsed_data

        except json.JSONDecodeError as e:
            logger.error(f"Failed to parse OpenAI JSON response: {e}")
            logger.error(f"Raw response: {content}")
            raise HTTPException(status_code=500, detail="Failed to parse OpenAI response as JSON")

    except Exception as e:
        logger.error(f"OpenAI API call failed: {e}")
        raise HTTPException(status_code=500, detail=f"OpenAI API error: {str(e)}")

def format_transaction_date(date_str):
    """
    Convert various date formats to ISO format (YYYY-MM-DD)
    Handles common receipt date formats
    """
    if not date_str or not isinstance(date_str, str):
        return None

    # Clean the input
    date_str = date_str.strip()

    try:
        # Pattern 1: DD/MM/YY (20/07/25)
        if re.match(r'^\d{1,2}/\d{1,2}/\d{2}$', date_str):
            day, month, year = date_str.split('/')
            full_year = f"20{year}" if int(year) <= 30 else f"19{year}"
            return f"{full_year}-{month.zfill(2)}-{day.zfill(2)}"

        # Pattern 2: DD/MM/YYYY (20/07/2025)
        elif re.match(r'^\d{1,2}/\d{1,2}/\d{4}$', date_str):
            day, month, year = date_str.split('/')
            return f"{year}-{month.zfill(2)}-{day.zfill(2)}"

        # Pattern 3: MM/DD/YY (07/20/25) - US format
        elif re.match(r'^\d{1,2}/\d{1,2}/\d{2}$', date_str):
            # This conflicts with DD/MM/YY - you'll need business logic to decide
            # For now, assuming DD/MM format is more common
            pass

        # Pattern 4: MM/DD/YYYY (07/20/2025) - US format
        elif re.match(r'^\d{1,2}/\d{1,2}/\d{4}$', date_str):
            # Same conflict as above
            pass

        # Pattern 5: DD-MM-YY (20-07-25)
        elif re.match(r'^\d{1,2}-\d{1,2}-\d{2}$', date_str):
            day, month, year = date_str.split('-')
            full_year = f"20{year}" if int(year) <= 30 else f"19{year}"
            return f"{full_year}-{month.zfill(2)}-{day.zfill(2)}"

        # Pattern 6: DD-MM-YYYY (20-07-2025)
        elif re.match(r'^\d{1,2}-\d{1,2}-\d{4}$', date_str):
            day, month, year = date_str.split('-')
            return f"{year}-{month.zfill(2)}-{day.zfill(2)}"

        # Pattern 7: DD.MM.YY (20.07.25)
        elif re.match(r'^\d{1,2}\.\d{1,2}\.\d{2}$', date_str):
            day, month, year = date_str.split('.')
            full_year = f"20{year}" if int(year) <= 30 else f"19{year}"
            return f"{full_year}-{month.zfill(2)}-{day.zfill(2)}"

        # Pattern 8: DD.MM.YYYY (20.07.2025)
        elif re.match(r'^\d{1,2}\.\d{1,2}\.\d{4}$', date_str):
            day, month, year = date_str.split('.')
            return f"{year}-{month.zfill(2)}-{day.zfill(2)}"

        # Pattern 9: YYYY-MM-DD (already ISO format)
        elif re.match(r'^\d{4}-\d{1,2}-\d{1,2}$', date_str):
            parts = date_str.split('-')
            return f"{parts[0]}-{parts[1].zfill(2)}-{parts[2].zfill(2)}"

        # Pattern 10: Text dates like "Jul 20, 2025" or "20 Jul 2025"
        elif re.search(r'[A-Za-z]{3}', date_str):
            # Try to parse with Python's datetime
            for fmt in ['%b %d, %Y', '%d %b %Y', '%B %d, %Y', '%d %B %Y',
                       '%b %d %Y', '%d-%b-%Y', '%d/%b/%Y']:
                try:
                    parsed = datetime.strptime(date_str, fmt)
                    return parsed.strftime('%Y-%m-%d')
                except ValueError:
                    continue

        # Pattern 11: DDMMYY (200725)
        elif re.match(r'^\d{6}$', date_str):
            day = date_str[:2]
            month = date_str[2:4]
            year = date_str[4:6]
            full_year = f"20{year}" if int(year) <= 30 else f"19{year}"
            return f"{full_year}-{month}-{day}"

        # Pattern 12: DDMMYYYY (20072025)
        elif re.match(r'^\d{8}$', date_str):
            day = date_str[:2]
            month = date_str[2:4]
            year = date_str[4:8]
            return f"{year}-{month}-{day}"

        # Pattern 13: Try Python's datetime parser as fallback
        else:
            for fmt in ['%d/%m/%Y', '%m/%d/%Y', '%Y/%m/%d', '%d-%m-%Y',
                       '%m-%d-%Y', '%Y-%m-%d', '%d.%m.%Y', '%m.%d.%Y']:
                try:
                    parsed = datetime.strptime(date_str, fmt)
                    return parsed.strftime('%Y-%m-%d')
                except ValueError:
                    continue

        # If nothing matches, return None
        return None

    except Exception as e:
        print(f"Error formatting date '{date_str}': {e}")
        return None

# =============================================================================
# API ENDPOINTS
# =============================================================================
@app.get("/")
async def root():
    """Health check endpoint"""
    return {
        "message": "Receipt OCR Parser API with User Management is running",
        "version": "1.0.0",
        "endpoints": [
            "/process-image",
            "/store-receipt (requires user_id)",
            "/search-receipts (optional user_id filter)",
            "/receipt/{receipt_id}",
            "/users/login",
            "/users/create",
            "/users/list",
            "/users/{user_id}/receipts",
            "/debug/users"
        ],
        "ocr_model": "PP-OCRv5_mobile",
        "ai_model": OPENAI_MODEL,
        "database": "SQLite with user associations",
        "supported_inputs": ["image_url", "image_base64"]
    }

@app.post("/process-image", response_model=ProcessImageResponse)
async def process_image(request: ProcessImageRequest):
    """
    Process receipt image using PaddleOCR Mobile + OpenAI

    Accepts either image_url or image_base64 (exactly one must be provided)
    """
    start_time = datetime.now()
    import time

    try:
        tik = time.time()
        # Validate input
        if not request.image_url and not request.image_base64:
            raise HTTPException(status_code=400, detail="Either image_url or image_base64 must be provided")

        if request.image_url and request.image_base64:
            raise HTTPException(status_code=400, detail="Provide either image_url OR image_base64, not both")

        # Initialize OCR if not already done
        ocr = initialize_ocr()

        # Load image based on input type
        if request.image_url:
            logger.info("Using image from URL")
            image_input = str(request.image_url)  # image_input = a string URL
        elif request.image_base64:
            logger.info("Using image from base64")
            image_input = load_image_from_base64(request.image_base64)  # image_input = np.ndarray or PIL.Image
        tok = time.time()
        print(f"\nLoad: Inference time: {tok-tik:.2f} seconds")

        # Perform OCR
        tik = time.time()
        logger.info("Running PaddleOCR...")
        ocr_result = ocr.predict(input=image_input)

        if not ocr_result or not ocr_result[0]:
            return ProcessImageResponse(
                success=False,
                error_message="No text detected in image",
                processing_time_seconds=(datetime.now() - start_time).total_seconds()
            )

        # Extract text and confidence scores
        rec_texts, rec_scores = ocr_result[0]['rec_texts'], ocr_result[0]['rec_scores']
        if not rec_texts:
            return ProcessImageResponse(
                success=False,
                error_message="No text extracted from OCR result",
                processing_time_seconds=(datetime.now() - start_time).total_seconds()
            )
        logger.info(f"OCR extracted {len(rec_texts)} text elements")
        tok = time.time()
        print(f"\nOCR: Inference time: {tok-tik:.2f} seconds")
        tik = time.time()

        # Create OpenAI prompt
        prompt = create_openai_prompt(rec_texts, rec_scores)

        # Call OpenAI API
        logger.info("Calling OpenAI API...")
        parsed_data = await call_openai_api(prompt)
        tok = time.time()
        print(f"\nOpenAI: Inference time: {tok-tik:.2f} seconds")
        tik = time.time()

        # Map confidence scores to extracted fields
        receipt_data = map_confidence_scores(parsed_data, rec_texts, rec_scores)
        tok = time.time()
        print(f"mapping: {tok-tik: .2f}")
        print(receipt_data)
        processing_time = (datetime.now() - start_time).total_seconds()
        avg_confidence = sum(rec_scores) / len(rec_scores) if rec_scores else 0.0

        logger.info(f"Processing completed successfully in {processing_time:.2f}s")

        return ProcessImageResponse(
            success=True,
            receipt_data=receipt_data,
            ocr_confidence_scores=rec_scores,
            average_confidence=avg_confidence,
            raw_ocr_text=rec_texts,
            processing_time_seconds=processing_time
        )

    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"Unexpected error: {e}")
        processing_time = (datetime.now() - start_time).total_seconds()
        return ProcessImageResponse(
            success=False,
            error_message=f"Internal server error: {str(e)}",
            processing_time_seconds=processing_time
        )

@app.post("/store-receipt", response_model=StoredReceiptResponse)
async def store_receipt(request: StoreReceiptRequest):
    """
    Store processed receipt data to SQLite database
    """
    try:
        receipt_id = store_receipt_to_db(
            user_id=request.user_id,
            receipt_data=request.receipt_data,
            ocr_confidence_scores=request.ocr_confidence_scores,
            raw_ocr_text=request.raw_ocr_text,
            average_confidence=request.average_confidence,
            processing_time_seconds=request.processing_time_seconds,
            image_url=request.image_url,
            notes=request.notes
        )

        return StoredReceiptResponse(
            success=True,
            receipt_id=receipt_id,
            message="Receipt stored successfully"
        )

    except Exception as e:
        logger.error(f"Failed to store receipt: {e}")
        return StoredReceiptResponse(
            success=False,
            message="Failed to store receipt",
            error_message=str(e)
        )

@app.post("/search-receipts", response_model=SearchReceiptsResponse)
async def search_receipts_endpoint(request: ReceiptSearchRequest):
    """
    Search stored receipts based on criteria
    """
    try:
        receipts = search_receipts(
            user_id=request.user_id,
            merchant_name=request.merchant_name,
            date_from=request.date_from,
            date_to=request.date_to,
            min_amount=request.min_amount,
            max_amount=request.max_amount,
            limit=request.limit
        )

        return SearchReceiptsResponse(
            success=True,
            receipts=receipts,
            count=len(receipts)
        )

    except Exception as e:
        logger.error(f"Failed to search receipts: {e}")
        return SearchReceiptsResponse(
            success=False,
            error_message=str(e)
        )

# Complete the update_receipt endpoint
@app.put("/receipt/{receipt_id}", response_model=UpdateReceiptResponse)
async def update_receipt(receipt_id: str, request: UpdateReceiptRequest):
    """
    Update a receipt by ID
    """
    try:
        # Validate receipt_id format (basic UUID check)
        if not receipt_id or len(receipt_id) < 10:
            raise HTTPException(status_code=400, detail="Invalid receipt ID format")
        print(f"HERE IS IT: ", request)
        # Update receipt in database
        success = update_receipt_in_db(
            receipt_id=receipt_id,
            user_id=request.user_id,
            receipt_data=request.receipt_data,
            notes=request.notes,
            image_url=request.image_url
        )

        if success:
            return UpdateReceiptResponse(
                success=True,
                message="Receipt updated successfully"
            )
        else:
            return UpdateReceiptResponse(
                success=False,
                message="Failed to update receipt",
                error_message="Unknown error occurred"
            )

    except ValueError as e:
        # Handle business logic errors (access denied, not found, etc.)
        return UpdateReceiptResponse(
            success=False,
            message="Update failed",
            error_message=str(e)
        )
    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"Failed to update receipt {receipt_id}: {e}")
        return UpdateReceiptResponse(
            success=False,
            message="Internal server error",
            error_message=str(e)
        )

@app.get("/receipt/{receipt_id}")
async def get_receipt_details(receipt_id: str, user_id: Optional[str] = None):
    """
    Get full receipt details by ID, optionally filtered by user for security
    """
    try:
        receipt_data = get_receipt_by_id(receipt_id)

        if not receipt_data:
            raise HTTPException(status_code=404, detail="Receipt not found")

        if user_id and receipt_data.get('user_id') != user_id:
            return {
                "success": False,
                "message": "Access denied - receipt belongs to different user"
            }

        return {
            "success": True,
            "receipt_data": receipt_data
        }

    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"Failed to get receipt details: {e}")
        raise HTTPException(status_code=500, detail=f"Internal server error: {str(e)}")

@app.get("/users/{user_id}/receipts")
async def get_user_receipts(user_id: str, limit: int = 50):
    """
    Get all receipts for a specific user
    """
    try:
        # Verify user exists
        with get_db_connection() as conn:
            cursor = conn.cursor()
            cursor.execute('SELECT user_id FROM users WHERE user_id = ? AND is_active = 1', (user_id,))
            if not cursor.fetchone():
                return {
                    "success": False,
                    "message": "User not found or inactive"
                }

        # Get user's receipts
        receipts = search_receipts(user_id=user_id, limit=limit)

        return {
            "success": True,
            "user_id": user_id,
            "receipts": receipts,
            "count": len(receipts)
        }

    except Exception as e:
        logger.error(f"Failed to get user receipts: {e}")
        return {
            "success": False,
            "message": "Internal server error",
            "error_message": str(e)
        }

@app.delete("/receipt/{receipt_id}", response_model=DeleteReceiptResponse)
async def delete_receipt(receipt_id: str, request: DeleteReceiptRequest):
    """
    Delete a receipt by ID
    """
    try:
        # Validate receipt_id format (basic UUID check)
        if not receipt_id or len(receipt_id) < 10:
            raise HTTPException(status_code=400, detail="Invalid receipt ID format")

        # Delete receipt from database
        success = delete_receipt_from_db(receipt_id=receipt_id, user_id=request.user_id)

        if success:
            return DeleteReceiptResponse(
                success=True,
                message="Receipt deleted successfully"
            )
        else:
            return DeleteReceiptResponse(
                success=False,
                message="Failed to delete receipt",
                error_message="Unknown error occurred"
            )

    except ValueError as e:
        # Handle business logic errors (access denied, not found, etc.)
        return DeleteReceiptResponse(
            success=False,
            message="Delete failed",
            error_message=str(e)
        )
    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"Failed to delete receipt {receipt_id}: {e}")
        return DeleteReceiptResponse(
            success=False,
            message="Internal server error",
            error_message=str(e)
        )
# =============================================================================
# USER ENDPOINTS
# =============================================================================

@app.post("/users/login", response_model=LoginResponse)
async def login_user(request: LoginRequest):
    """Authenticate user (no session management for now)"""
    try:
        user_data = authenticate_user(request.username, request.password)

        if not user_data:
            return LoginResponse(
                success=False,
                message="Invalid username or password"
            )

        user = User(
            user_id=user_data['user_id'],
            username=user_data['username'],
            email=user_data['email'],
            full_name=user_data.get('full_name'),
            role=user_data['role'],
            is_active=bool(user_data['is_active']),
            created_at=user_data.get('created_at', ''),
            last_login=user_data.get('last_login')
        )

        return LoginResponse(
            success=True,
            message="Login successful",
            user=user
        )

    except Exception as e:
        logger.error(f"Login failed: {e}")
        return LoginResponse(
            success=False,
            message="Login failed",
            error_message=str(e)
        )

@app.post("/users/create", response_model=UserResponse)
async def create_user_endpoint(request: CreateUserRequest):
    """Create a new user (no auth required for now - make it admin-only later)"""
    try:
        # For now, anyone can create users. Add auth later.
        user_id = create_user(
            username=request.username,
            email=request.email,
            full_name=request.full_name,
            password=request.password,
            role=request.role
        )

        # Get created user data
        user_data = get_user_by_id(user_id)

        user = User(
            user_id=user_data['user_id'],
            username=user_data['username'],
            email=user_data['email'],
            full_name=user_data.get('full_name'),
            role=user_data['role'],
            is_active=bool(user_data['is_active']),
            created_at=user_data['created_at'],
            last_login=user_data.get('last_login')
        )

        return UserResponse(
            success=True,
            user=user,
            message="User created successfully"
        )

    except ValueError as e:
        return UserResponse(
            success=False,
            message="Failed to create user",
            error_message=str(e)
        )
    except Exception as e:
        logger.error(f"Failed to create user: {e}")
        return UserResponse(
            success=False,
            message="Failed to create user",
            error_message=str(e)
        )

@app.get("/users/list", response_model=UsersListResponse)
async def list_users():
    """List all users (no auth required for now - make it admin-only later)"""
    try:
        users_data = get_all_users()

        users = []
        for user_data in users_data:
            users.append(User(
                user_id=user_data['user_id'],
                username=user_data['username'],
                email=user_data['email'],
                full_name=user_data.get('full_name'),
                role=user_data['role'],
                is_active=bool(user_data['is_active']),
                created_at=user_data['created_at'],
                last_login=user_data.get('last_login')
            ))

        return UsersListResponse(
            success=True,
            users=users,
            count=len(users)
        )

    except Exception as e:
        logger.error(f"Failed to list users: {e}")
        return UsersListResponse(
            success=False,
            error_message=str(e)
        )

def run_server():
    """Run the FastAPI server with ngrok"""
    try:
        # Setup ngrok tunnel
        public_url = setup_ngrok()

        # Run the FastAPI server
        uvicorn.run(
            app,
            host="0.0.0.0",
            port=8000,
            log_level="info"
        )

    except Exception as e:
        logger.error(f"Failed to start server: {e}")
        raise

@app.on_event("startup")
async def startup_event():
    """Initialize OCR and database on startup"""
    initialize_ocr()
    initialize_database()
    add_user_id_column_if_missing()  # ADD THIS LINE
    create_default_admin_user()  # ADD THIS IF YOU WANT DEFAULT ADMIN

# Main execution
if __name__ == "__main__":
    run_server()